In [ ]:
# Import necessari
import cv2
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import json
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
import os
import glob
from PIL import Image
import os
import glob
from PIL import Image

class ParkingLotDataset(Dataset):
    def __init__(self, root_img, root_msk, pairs=None, transforms=None, mask_transforms=None):
        self.root_img = root_img
        self.root_msk = root_msk
        self.transforms = transforms
        self.mask_transforms = mask_transforms

        if pairs is None:
            # Get all image files
            self.image_paths = sorted(glob.glob(os.path.join(root_img, '*.png')))

            # Get all mask files
            self.mask_paths = sorted(glob.glob(os.path.join(root_msk, '*.png')))

            # Pair image and mask files based on their filenames
            #self.pairs = [(image_path, mask_path) for image_path in self.image_paths for mask_path in self.mask_paths if os.path.splitext(os.path.basename(image_path))[0] == os.path.splitext(os.path.basename(mask_path))[0]]
            self.pairs = []

            for image_path in self.image_paths:
                image_filename = os.path.splitext(os.path.basename(image_path))[0]
                mask_filename = f"{image_filename}_SegmentationClass.png"
                mask_path = os.path.join(root_msk, mask_filename)
                if os.path.exists(mask_path):
                    self.pairs.append((image_path, mask_path))

        else:
            self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx, threshold=0.5):
        image_path, mask_path = self.pairs[idx]

        # Load image
        image = Image.open(image_path)
        image_array = np.array(image)
        self.input_channels = image_array.shape[0]

        # Apply transformations
        if self.transforms:
            image_array = self.transforms(image_array)
        
        #mask = Image.open(mask_path)
        #mask_array = np.array(mask)
        mask_array = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask_array = (mask_array > threshold).astype(np.float32)
        self.input_channels = mask_array.shape[0]

        # Apply transformations
        if self.mask_transforms:
            mask_array = self.mask_transforms(mask_array)


        return image_array, mask_array
        


In [ ]:
import random
from torch.utils.data import random_split
from torchvision.transforms import functional as F

# magari rifare il dataloader con due cartelle
train_path = '/kaggle/input/big-ddd/Dataset_splittato/train_images'
train_mask_path = '/kaggle/input/big-ddd/Dataset_splittato/train_masks'

val_path = '/kaggle/input/big-ddd/Dataset_splittato/val_images'
val_mask_path = '/kaggle/input/big-ddd/Dataset_splittato/val_masks'

test_path = '/kaggle/input/big-ddd/Dataset_splittato/test_images'
test_mask_path = '/kaggle/input/big-ddd/Dataset_splittato/test_masks'

#normalize = transforms.Normalize(mean=[0.5], std=[0.5])
#normalize = transforms.Normalize(mean=[35.5, 35.2, 33.4], std=[21.8, 21.6, 20.9])

transform = transforms.Compose([
    
    transforms.ToTensor(),
    
    #transforms.Normalize(mean=[35.5, 35.2, 33.4], std=[21.8, 21.6, 20.9]),
    # Add other transforms here as needed
])

mask_transforms = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.5], std=[0.5]),
    # Add other mask transformations here
])



# Create datasets for each split
train_dataset = ParkingLotDataset(train_path, train_mask_path, transforms=transform, mask_transforms=mask_transforms)
val_dataset = ParkingLotDataset(val_path, val_mask_path, transforms=transform, mask_transforms=mask_transforms)
test_dataset = ParkingLotDataset(test_path, test_mask_path, transforms=transform, mask_transforms=mask_transforms)

# Now you can create data loaders for each split
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False) 

In [ ]:
from torch.utils.data import DataLoader
from torch import optim
import torch.nn.functional as F
import wandb
from torch.optim.lr_scheduler import StepLR
from torch import save
from torch.optim.lr_scheduler import ReduceLROnPlateau
import segmentation_models_pytorch as smp
wandb.login(key='cf05b564865bb4bf8601ed59cbace5b02a587fa9')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the pretrained U-Net model
model = smp.Unet(
    encoder_name="resnet34",  # Choose the encoder (backbone)
    encoder_weights="imagenet",  # Use pre-trained weights from ImageNet
    in_channels=3,  # Input channels (RGB)
    classes=1,  # Binary segmentation
)

# Move the model to the device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 10
run = wandb.init(
    #Set the project where this run will be logged
    project="Parking_lot_zones",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": 0.01,
        "epochs": epochs,
    },
    #entity='lorenzo_barbieri'
    entity='occelli-2127855'
)

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device, dtype=torch.float32)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    wandb.log({"Train Loss": train_loss})

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device, dtype=torch.float32)

            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    wandb.log({"Validation Loss": val_loss})

    print(f"Epoch {epoch+1}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

In [ ]:
import torch
import matplotlib.pyplot as plt


model.eval()

# Testing loop
total = 0
correct = 0
batch_idx = 0
with torch.no_grad():
    for batch in test_loader:
        images, masks = batch[:2]  # Unpack the first two values (images and masks)
        images = images.to(device)
        masks = masks.to(device, dtype=torch.float32)
        outputs = model(images)

        # Threshold the outputs to obtain binary masks
        predicted = (outputs > 0.5).float()

        # Compute the accuracy
        total += masks.numel()  # Total number of elements in the batch
        correct += (predicted == masks).sum().item()  # Correctly predicted elements

        # Plot the first 5 outputs
        if batch_idx < 14:
            plt.figure(figsize=(10, 4))
            plt.subplot(1, 3, 1)
            plt.imshow(images[0].permute(1, 2, 0).cpu().numpy())
            plt.title('Input Image')
            plt.subplot(1, 3, 2)
            plt.imshow(masks[0][0].cpu(), cmap='gray')
            plt.title('Ground Truth Mask')
            plt.subplot(1, 3, 3)
            plt.imshow(predicted[0][0].cpu(), cmap='gray')
            plt.title('Predicted Mask')
            plt.show()
        batch_idx += 1

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")
wandb.log({"Test Accuracy": accuracy})
wandb.finish()